# 声明模块接口

在声明模块接口方面,主要是3个方面:

+ 声明类型别名
+ 声明C全局变量
+ 声明C函数
+ 声明扩展类型

尤其是结合纯净模式,我们可以做到渐进式的优化模块性能.

这块的演示代码在[]()中

In [1]:
%load_ext cython

## `.pxd`文件的编译时行为

在编译时cython会自动在指定的实现文件同目录寻找与实现部分源码文件同名的声明文件共同编译.因此我们在编译时并不需要显式的指定`.pxd`文件,指定`.py`或`.pyx`文件即可.

如果找到了声明文件,cython会匹配声明文件和实现文件,并校验声明的内容是否能在实现部分找到.这包括

1. xxx
2. 声明的函数在实现部分存不存在,函数签名是否匹配
3. 声明的扩展类型在实现部分是否存在,其中的方法签名是否匹配

如果都没有问题,cython就会正式进行编译.

我个人推荐设置如下编译设置项:

```cython
# cython: embedsignature=True
# cython: embedsignature.format=python
```

这样设置后,声明文件中设置的类型信息会被cython写入对应python可调用对象的说明文档.这样方便调用方查看使用.

## 声明类型别名

Cython中我们可以使用关键词`ctypedef`来声明类型别名,其语法为`ctypedef 类型 类型别名`,比如将`unsigned int`定义为`uint_t`可以写作

+ `A.pxd`

    ```cython
    ctypedef unsigned int uint_t
    ```

`ctypedef`可以也仅可以用于任何**C类型**,例如整数,浮点数,struct,指针等.通常用于提高可读性和对模块使用场景的匹配程度.

通常别名声明既可以放在`.pxd`文件也可以放在实现部分,这取决于如下几点:

1. 你是否希望外部cython模块使用这个别名,如果希望则放在`.pxd`文件,不希望则放实现部分
2. 你的对外接口是否有用到这个别名,如果有用到则放在`.pxd`文件,没有则放实现部分

一旦你在`.pxd`文件中声明了别名,那实现部分就不需要重复声明了.

## 声明C变量

你可以直接用`cdef 类型 变量名`声明一个变量,比如

+ `A.pxd`

    ```cython
    cdef int GLOBAL_VAL
    ```

通常变量声明既可以放在`.pxd`文件也可以放在实现部分,这取决于你是否希望外部cython模块使用这个变量,如果希望则放在`.pxd`文件,不希望则放实现部分.

一旦你在.pxd文件中声明变量,那实现部分就不需要重复声明了可以直接赋值.比如上面声明文件对应的实现文件中可以直接这样赋值

+ `A.pyx`

    ```cython
    GLOBAL_VAL = 100
    ```

## 声明C函数

我们可以使用`cdef [返回值类型] 函数名(形参类型 形参名,...) [noexcept | except 异常标志量]`来声明一个C函数;`cpdef 返回值类型 函数名(形参类型 形参,...)`来声明一个Python和C都可调用的函数

需要注意的是如果参数有默认值,这个默认值在实现部分设置,但`.pxd`文件的声明中要使用`=*`来表示这个参数有默认值

+ `A.pxd`

    ```cython
    cdef uint_t mycfunc(uint_t x, uint_t y=*)

    cpdef int mycpfunc(int x, int y=*)
    ```
    
+ `A.pyx`

    ```cython
    cdef uint_t mycfunc(uint_t x, uint_t y=2):
        cdef uint_t a
        a = x-y
        return a + x * y

    cpdef int mycpfunc(int x, int y=2):
        cdef int a
        a = x+y
        return a + x * y

    ```
    
**特别需要注意的是**:python函数不能在`.pxd`中声明,**包括静态化参数的python函数**.

通常函数声明既可以放在`.pxd`文件也可以放在实现部分,这取决于你是否希望外部cython模块使用这个函数,如果希望则放在.pxd文件,不希望则放实现部分.

### 声明内联函数

在`.pyd`文件中同样可以写内联函数.但要注意内联函数除了要声明还要有实现

```cython
cdef inline int int_min(int a, int b):
    return b if b < a else a
```

## 声明扩展类型

扩展类型中只需要且只能声明其中的C方法,Python和C都可调用的方法以及C属性,python部分不能声明.

+ `A.pxd`

```cython
cdef class A:
    cdef public int a
    cdef int b
    cdef double foo(self,double x)
    cpdef double bar(self, double x)
```

其中C属性部分在`.pxd`中声明了就不需要再在实现部分声明了,只要正常赋值即可.

+ `A.pyx`

```cython
cdef class A:
    def __init__(self, b:int = 0):
        self.a = 3
        self.b = b

    cdef double foo(self, double x):
        return x + _helper(1.0)
    
    cpdef double bar(self, double x):
        return x**2 + _helper(1.0)

    def foobar(self,x:float)->float:
        return x**2 + _helper(1.0)
```


## 与纯净模式结合使用

普通cython语法具有更好的表达能力,代码可读性更好功能也最全.但纯净模式与声明文件`.pyd`结合使用更加适合渐进式的性能优化工作流.我们可以先使用纯python实现,然后根据需要在对原始python代码做出极小改动的情况下使用声明文件`.pyd`将其改造为cython模块.

cython在编译时会将声明文件`.pyd`中声明的变量,函数,方法,等与源文件中进行匹配,在纯净模式下,由于有声明文件`.pyd`中的声明,即便实现文件中定义的是python函数python类,也可以被当做声明成的C函数扩展类来进行编译.

这块的演示代码在[]()中

### 判别当前模块是否被编译过

当作为python模块使用时,与纯净模式结合使用的一个问题是我们不能直观看出调用的是`.py`文件还是编译好的动态连接库.我们可以使用`cython.compiled`这个cython的全局变量来进行判断

In [2]:
import cython

if cython.compiled:
    print("Yep, I'm compiled.")
    
else:
    print("Just a lowly interpreted script.")


Just a lowly interpreted script.


In [4]:
%%cython

import cython

if cython.compiled:
    print("Yep, I'm compiled.")
    
else:
    print("Just a lowly interpreted script.")

Yep, I'm compiled.


我们通常会在纯净模式的源码中开头写上上面的判断并在其中写一些专用的代码逻辑

### 纯净模式改写原例子 

我们只需要改写实现部分:

+ `A.py`

```cython
# cython: embedsignature=True
# cython: embedsignature.format=python
import cython

if cython.compiled:
    print("Yep, I'm compiled.")

else:
    print("Just a lowly interpreted script.")
    uint_t = int
    def int_min(a: int, b: int) -> int:
        return b if b < a else a

GLOBAL_VAL = 100

# uint_t = cython.typedef(cython.uint)


def mycfunc(x: uint_t, y: uint_t = 2) -> uint_t:
    a: uint_t = x - y
    return a + x * y


def mycpfunc(x: cython.int, y: cython.int = 2) -> int:
    a: cython.int = x + y
    return a + x * y


@cython.locals(x=cython.int, y=cython.int)
def mypfuncstatic(x: int, y: int) -> int:
    return x * y


def mypfunc(x: int, y: int) -> int:
    return 2 * x * y


def _helper(a: cython.double) -> float:
    return a + 1


class A:
    def __init__(self, b: int = 0):
        self.a = 3
        self.b = b

    def foo(self, x: cython.double) -> cython.double:
        return x + _helper(1.0)

    def bar(self, x: float) -> float:
        return x**2 + _helper(1.0)

    def foobar(self, x: float) -> float:
        return x**2 + _helper(1.0)
```

通常用纯净模式都是要考虑不编译情况下纯python代码的可执行性的,因此需要注意的有如下几点:

1. 声明文件中的类型别名在cython中可以被直接使用,但在python下不行,需要像例子中一样在检查到没有编译的情况下提前声明别名
2. 推荐无论是C函数,python函数还是C和python都可以调用的函数都保留typehints方便维护
3. 建议C函数,C和python都可以调用的函数在typehints中使用`cython`模块中的类型进行声明;不静态化参数的python函数在typehints中使用python;静态化参数的python函数必须在typehints中使用`cython`模块中的类型进行声明
4. 在给python写扩展的情况下建议非python接口的函数以及方法遵循python社区西关使用`_`开头命名